# Minimal adversarial attack example
## with cleverhans and the iris dataset
#### by August Karlstedt
Most adversarial examples are shown in images where a normal, unmodified image is displayed as the input. An adversarial noise is crafted, and is shown being added to the original input image. The reader can then see how this noise modifies the image in a nearly unnoticeable way, while the classifier output is changed significantly.

While this is good for a visual understanding, it complicates some things when thinking about the actual happenings of adversarial attacks. Therefore, I have constructed a simple example using the infamous iris dataset. One can clearly see the small changes to the input will change the predicted class. It is easy to modify parameters and see how it affects the adversarial algorithm.

In [1]:
import glob

import numpy as np
import pandas as pd
import tensorflow as tf

# Set TF random seed to improve reproducibility
tf.set_random_seed(1234)
np.random.seed(1234)

In [2]:
file = 'data/bezdekIris.data'

data = pd.read_csv(file, names=['sepal length', 'sepal width', 'petal length', 'petal width', 'class'])
data['sepal length'] = data['sepal length'].astype(np.float32)
data['sepal width'] = data['sepal width'].astype(np.float32)
data['petal length'] = data['petal length'].astype(np.float32)
data['petal width'] = data['petal width'].astype(np.float32)
data.describe().T

,count,mean,std,min,25%,50%,75%,max
sepal length,150.0,5.843333,0.828066,4.3,5.1,5.80,6.4,7.9
sepal width,150.0,3.057333,0.435866,2.0,2.8,3.00,3.3,4.4
petal length,150.0,3.758000,1.765298,1.0,1.6,4.35,5.1,6.9
petal width,150.0,1.199333,0.762238,0.1,0.3,1.30,1.8,2.5


Our values match what is provided in `data/iris.names`

In [3]:
data.sample(10)

,sepal length,sepal width,petal length,petal width,class
91,6.1,3.0,4.6,1.4,Iris-versicolor
63,6.1,2.9,4.7,1.4,Iris-versicolor
103,6.3,2.9,5.6,1.8,Iris-virginica
6,4.6,3.4,1.4,0.3,Iris-setosa
59,5.2,2.7,3.9,1.4,Iris-versicolor
29,4.7,3.2,1.6,0.2,Iris-setosa
27,5.2,3.5,1.5,0.2,Iris-setosa
35,5.0,3.2,1.2,0.2,Iris-setosa
99,5.7,2.8,4.1,1.3,Iris-versicolor
122,7.7,2.8,6.7,2.0,Iris-virginica


In [4]:
x_train = data[['sepal length', 'sepal width', 'petal length', 'petal width']]
x_train = x_train.values

In [5]:
unique_classes = data['class'].unique().tolist()
unique_classes

['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

In [6]:
class_count = len(unique_classes)
class_count

3

In [7]:
class_to_int = {}
for i, item in enumerate(unique_classes):
    class_to_int[item] = i
class_to_int

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

In [8]:
def int_to_class(i):
    return unique_classes[i]
int_to_class(1)

'Iris-versicolor'

In [9]:
y_index = []
y_onehot = []

for item in data[['class']].values:
    item = item[0]
    
    vec = np.zeros(class_count)
    index = class_to_int[item]
    vec[index] = 1
    
    y_index.append(index)
    y_onehot.append(vec)

data['index'] = y_index
data['onehot'] = y_onehot

In [10]:
data.sample(10)

,sepal length,sepal width,petal length,petal width,class,index,onehot
76,6.8,2.8,4.8,1.4,Iris-versicolor,1,"[0.0, 1.0, 0.0]"
105,7.6,3.0,6.6,2.1,Iris-virginica,2,"[0.0, 0.0, 1.0]"
73,6.1,2.8,4.7,1.2,Iris-versicolor,1,"[0.0, 1.0, 0.0]"
117,7.7,3.8,6.7,2.2,Iris-virginica,2,"[0.0, 0.0, 1.0]"
115,6.4,3.2,5.3,2.3,Iris-virginica,2,"[0.0, 0.0, 1.0]"
128,6.4,2.8,5.6,2.1,Iris-virginica,2,"[0.0, 0.0, 1.0]"
15,5.7,4.4,1.5,0.4,Iris-setosa,0,"[1.0, 0.0, 0.0]"
18,5.7,3.8,1.7,0.3,Iris-setosa,0,"[1.0, 0.0, 0.0]"
55,5.7,2.8,4.5,1.3,Iris-versicolor,1,"[0.0, 1.0, 0.0]"
77,6.7,3.0,5.0,1.7,Iris-versicolor,1,"[0.0, 1.0, 0.0]"


In [11]:
y_train = np.array(y_onehot)

In [12]:
from cleverhans.attacks import FastGradientMethod
from cleverhans.augmentation import random_horizontal_flip, random_shift
from tensorflow.compat.v1 import flags
from cleverhans.dataset import CIFAR10
from cleverhans.loss import CrossEntropy
from cleverhans import initializers
from cleverhans.model import Model
from cleverhans.train import train
from cleverhans.utils import AccuracyReport, set_log_level
from cleverhans.utils_tf import model_eval

W1020 22:15:00.960482 20892 deprecation_wrapper.py:119] From c:\users\august\appdata\local\programs\python\python36\lib\site-packages\cleverhans\utils_tf.py:341: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.



In [13]:
NB_EPOCHS = 128
BATCH_SIZE = 16
LEARNING_RATE = 0.001
CLEAN_TRAIN = True
BACKPROP_THROUGH_ATTACK = False

In [14]:
report = AccuracyReport()

In [15]:
class BasicModel(Model):
    def __init__(self, scope, nb_classes, **kwargs):
        del kwargs
        Model.__init__(self, scope, nb_classes, locals())

        # Do a dummy run of fprop to create the variables from the start
        self.fprop(tf.placeholder(tf.float32, (1, 4)))
        # Put a reference to the params in self so that the params get pickled
        self.params = self.get_params()

    def fprop(self, x, **kwargs):
        del kwargs

        y = x

        with tf.variable_scope(self.scope, reuse=tf.AUTO_REUSE):
            y = tf.layers.Dense(4)(y) 
            y = tf.layers.Dense(32, activation='relu')(y)
            logits = tf.layers.Dense(self.nb_classes)(y)
    
        return {self.O_LOGITS: logits, self.O_PROBS: tf.nn.softmax(logits=logits)}

In [16]:
model = BasicModel('example-model', class_count)

W1020 22:15:00.999475 20892 deprecation.py:506] From c:\users\august\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1020 22:15:01.227511 20892 deprecation_wrapper.py:119] From c:\users\august\appdata\local\programs\python\python36\lib\site-packages\cleverhans\model.py:129: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.



In [17]:
sess = tf.Session()

In [18]:
x = tf.placeholder(tf.float32, shape=(None, 4))
y = tf.placeholder(tf.float32, shape=(None, class_count))

# Train an MNIST model
train_params = {
  'nb_epochs': NB_EPOCHS,
  'batch_size': BATCH_SIZE,
  'learning_rate': LEARNING_RATE
}
eval_params = {'batch_size': BATCH_SIZE}
fgsm_params = {
  'eps': 0.99,
  'clip_min': 0.,
  'clip_max': 10.
}
rng = np.random.RandomState([2019, 10, 20])

In [19]:
label_smoothing = 0.0

In [20]:
preds = model.get_logits(x)
loss = CrossEntropy(model, smoothing=label_smoothing)

In [21]:
x_test = x_train
y_test = y_train

def do_eval(preds, x_set, y_set, report_key, is_adv=None):
    acc = model_eval(sess, x, y, preds, x_set, y_set, args=eval_params)
    setattr(report, report_key, acc)
    if is_adv is None:
      report_text = None
    elif is_adv:
      report_text = 'adversarial'
    else:
      report_text = 'legitimate'
    if report_text:
      print('Test accuracy on %s examples: %0.4f' % (report_text, acc))
    
def evaluate():
    do_eval(preds, x_test, y_test, 'clean_train_clean_eval', False)

In [22]:
train(sess, loss, x_train, y_train, evaluate=evaluate, args=train_params, rng=rng, var_list=model.get_params())

c:\users\august\appdata\local\programs\python\python36\lib\site-packages\cleverhans\utils_tf.py:511: UserWarning: No GPUS, running on CPU
  warnings.warn("No GPUS, running on CPU")
W1020 22:15:01.315477 20892 deprecation_wrapper.py:119] From c:\users\august\appdata\local\programs\python\python36\lib\site-packages\cleverhans\train.py:117: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W1020 22:15:01.315477 20892 deprecation_wrapper.py:119] From c:\users\august\appdata\local\programs\python\python36\lib\site-packages\cleverhans\train.py:137: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1020 22:15:01.333476 20892 deprecation.py:506] From c:\users\august\appdata\local\programs\python\python36\lib\site-packages\cleverhans\compat.py:124: calling softmax_cross_entropy_with_logits_v2_helper (from tensorflow.python.ops.nn_ops) with dim is deprecated and will be removed in a future version.
Instruction

num_devices:  1
Test accuracy on legitimate examples: 0.3333
Test accuracy on legitimate examples: 0.2800


I1020 22:15:01.583477 20892 train.py:257] Epoch 2 took 0.005001544952392578 seconds
[INFO 2019-10-20 22:15:01,593 cleverhans] Epoch 3 took 0.004997968673706055 seconds
I1020 22:15:01.593477 20892 train.py:257] Epoch 3 took 0.004997968673706055 seconds
[INFO 2019-10-20 22:15:01,604 cleverhans] Epoch 4 took 0.005999565124511719 seconds
I1020 22:15:01.604475 20892 train.py:257] Epoch 4 took 0.005999565124511719 seconds
[INFO 2019-10-20 22:15:01,615 cleverhans] Epoch 5 took 0.005998849868774414 seconds
I1020 22:15:01.615475 20892 train.py:257] Epoch 5 took 0.005998849868774414 seconds
[INFO 2019-10-20 22:15:01,626 cleverhans] Epoch 6 took 0.00601649284362793 seconds
I1020 22:15:01.626494 20892 train.py:257] Epoch 6 took 0.00601649284362793 seconds
[INFO 2019-10-20 22:15:01,637 cleverhans] Epoch 7 took 0.006004810333251953 seconds
I1020 22:15:01.637480 20892 train.py:257] Epoch 7 took 0.006004810333251953 seconds
[INFO 2019-10-20 22:15:01,649 cleverhans] Epoch 8 took 0.005994081497192383 se

Test accuracy on legitimate examples: 0.3333
Test accuracy on legitimate examples: 0.3333
Test accuracy on legitimate examples: 0.3333
Test accuracy on legitimate examples: 0.3333
Test accuracy on legitimate examples: 0.3467
Test accuracy on legitimate examples: 0.6667
Test accuracy on legitimate examples: 0.6667
Test accuracy on legitimate examples: 0.6667
Test accuracy on legitimate examples: 0.6667
Test accuracy on legitimate examples: 0.6667
Test accuracy on legitimate examples: 0.7133
Test accuracy on legitimate examples: 0.7133
Test accuracy on legitimate examples: 0.8133
Test accuracy on legitimate examples: 0.7200
Test accuracy on legitimate examples: 0.8333
Test accuracy on legitimate examples: 0.7200
Test accuracy on legitimate examples: 0.9133


I1020 22:15:01.770477 20892 train.py:257] Epoch 19 took 0.006000041961669922 seconds
[INFO 2019-10-20 22:15:01,781 cleverhans] Epoch 20 took 0.005017995834350586 seconds
I1020 22:15:01.781500 20892 train.py:257] Epoch 20 took 0.005017995834350586 seconds
[INFO 2019-10-20 22:15:01,791 cleverhans] Epoch 21 took 0.0049970149993896484 seconds
I1020 22:15:01.791474 20892 train.py:257] Epoch 21 took 0.0049970149993896484 seconds
[INFO 2019-10-20 22:15:01,803 cleverhans] Epoch 22 took 0.005002498626708984 seconds
I1020 22:15:01.803478 20892 train.py:257] Epoch 22 took 0.005002498626708984 seconds
[INFO 2019-10-20 22:15:01,815 cleverhans] Epoch 23 took 0.005991697311401367 seconds
I1020 22:15:01.815475 20892 train.py:257] Epoch 23 took 0.005991697311401367 seconds
[INFO 2019-10-20 22:15:01,827 cleverhans] Epoch 24 took 0.0049746036529541016 seconds
I1020 22:15:01.827477 20892 train.py:257] Epoch 24 took 0.0049746036529541016 seconds
[INFO 2019-10-20 22:15:01,837 cleverhans] Epoch 25 took 0.004

...

I1020 22:15:02.707476 20892 train.py:257] Epoch 102 took 0.005990028381347656 seconds
[INFO 2019-10-20 22:15:02,719 cleverhans] Epoch 103 took 0.005000114440917969 seconds
I1020 22:15:02.719476 20892 train.py:257] Epoch 103 took 0.005000114440917969 seconds
[INFO 2019-10-20 22:15:02,730 cleverhans] Epoch 104 took 0.005999088287353516 seconds
I1020 22:15:02.730474 20892 train.py:257] Epoch 104 took 0.005999088287353516 seconds
[INFO 2019-10-20 22:15:02,741 cleverhans] Epoch 105 took 0.005013227462768555 seconds
I1020 22:15:02.741491 20892 train.py:257] Epoch 105 took 0.005013227462768555 seconds
[INFO 2019-10-20 22:15:02,752 cleverhans] Epoch 106 took 0.004999399185180664 seconds
I1020 22:15:02.752476 20892 train.py:257] Epoch 106 took 0.004999399185180664 seconds
[INFO 2019-10-20 22:15:02,762 cleverhans] Epoch 107 took 0.004999399185180664 seconds
I1020 22:15:02.762475 20892 train.py:257] Epoch 107 took 0.004999399185180664 seconds
[INFO 2019-10-20 22:15:02,774 cleverhans] Epoch 108 to

Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9667
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733


[INFO 2019-10-20 22:15:02,900 cleverhans] Epoch 119 took 0.005998849868774414 seconds
I1020 22:15:02.900477 20892 train.py:257] Epoch 119 took 0.005998849868774414 seconds
[INFO 2019-10-20 22:15:02,910 cleverhans] Epoch 120 took 0.00500178337097168 seconds
I1020 22:15:02.910478 20892 train.py:257] Epoch 120 took 0.00500178337097168 seconds
[INFO 2019-10-20 22:15:02,922 cleverhans] Epoch 121 took 0.0059967041015625 seconds
I1020 22:15:02.922475 20892 train.py:257] Epoch 121 took 0.0059967041015625 seconds
[INFO 2019-10-20 22:15:02,934 cleverhans] Epoch 122 took 0.00600123405456543 seconds
I1020 22:15:02.934479 20892 train.py:257] Epoch 122 took 0.00600123405456543 seconds
[INFO 2019-10-20 22:15:02,947 cleverhans] Epoch 123 took 0.006032466888427734 seconds
I1020 22:15:02.947476 20892 train.py:257] Epoch 123 took 0.006032466888427734 seconds
[INFO 2019-10-20 22:15:02,959 cleverhans] Epoch 124 took 0.005999326705932617 seconds
I1020 22:15:02.959476 20892 train.py:257] Epoch 124 took 0.005

Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9667
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9733
Test accuracy on legitimate examples: 0.9667
Test accuracy on legitimate examples: 0.9733


True

In [23]:
fgsm = FastGradientMethod(model, sess=sess)
adv_x = fgsm.generate(x, **fgsm_params)
preds_adv = model.get_logits(adv_x)

W1020 22:15:03.044509 20892 deprecation.py:323] From c:\users\august\appdata\local\programs\python\python36\lib\site-packages\cleverhans\attacks\__init__.py:283: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


In [24]:
random_example = data.sample(1)
random_example

,sepal length,sepal width,petal length,petal width,class,index,onehot
7,5.0,3.4,1.5,0.2,Iris-setosa,0,"[1.0, 0.0, 0.0]"


In [25]:
example_x = random_example[['sepal length', 'sepal width', 'petal length', 'petal width']]
example_x

,sepal length,sepal width,petal length,petal width
7,5.0,3.4,1.5,0.2


In [26]:
example_y = random_example['onehot']
example_y

7    [1.0, 0.0, 0.0]
Name: onehot, dtype: object

First, let's run the normal model:

In [27]:
sess.run(model.get_logits(x), feed_dict={x: example_x })

array([[  4.6078844,  -0.6497631, -12.955629 ]], dtype=float32)

and the probabilities:

In [28]:
probs = sess.run(model.get_probs(x), feed_dict={x: example_x })
probs

array([[9.9481946e-01, 5.1805628e-03, 2.3442652e-08]], dtype=float32)

In [29]:
int_to_class(probs.argmax())

'Iris-setosa'

Now, let's see what adversarial perturbation is going to be made:

In [30]:
sess.run(adv_x, feed_dict={x: example_x })

array([[4.01, 2.41, 2.49, 1.19]], dtype=float32)

And if we check the logits:

In [31]:
sess.run(preds_adv, feed_dict={x: example_x })

array([[-1.964462  ,  0.51600164, -3.5647297 ]], dtype=float32)

and the output probabilities:

In [32]:
probs = sess.run(model.get_probs(adv_x), feed_dict={x: example_x })
probs

array([[0.07605346, 0.9085958 , 0.01535082]], dtype=float32)

In [33]:
int_to_class(probs.argmax())

'Iris-versicolor'

we can see that they were perturbed by